# Installing the modules and using my developer app to interact with the API

In [1]:
!pip install spotipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 562.1 kB/s eta 0:00:0000:0100:01


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id='66a8ab5007af449380b31c6c613c708c', client_secret='abbc64b7ed414636868764acf1f5b566')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

# Using the API to scrape the song information (name, artist, metrics etc)

In [4]:
#pasting the link to my playlist
playlist_link = "https://open.spotify.com/playlist/6qVWmafDIRhmjY8uBYOQ8U"
#line of code which gets the URI
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
#line of code to get the URI for each track in the playlist
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [16]:
# Creating empty lists to store information
track_uris = []
track_names = []
artist_names = []
artist_genres = []
track_popularities = []

# Initial request for the first page of tracks
results = sp.playlist_tracks(playlist_URI)

# Iterate through all pages of tracks
while results["items"]:
    tracks = results["items"]
    for track in tracks:
        track_uris.append(track["track"]["uri"])
        track_names.append(track["track"]["name"])
        artist_names.append(track["track"]["artists"][0]["name"])

        artist_uri = track["track"]["artists"][0]["uri"]
        artist_info = sp.artist(artist_uri)
        artist_genres.append(artist_info["genres"])

        track_popularities.append(track["track"]["popularity"])

    # Checking to see if there more pages to retrieve as the Spotify API returns a limited number of tracks per request (100)
    if results["next"]:
        results = sp.next(results)  # Move to the next page of results
    else:
        break

# Create a DataFrame using the extracted information
data = {
    "Track URI": track_uris,
    "Track Name": track_names,
    "Artist Name": artist_names,
    "Artist Genres": artist_genres,
    "Track Popularity": track_popularities
}

df = pd.DataFrame(data)

# Print the DataFrame
print(df)


                                 Track URI           Track Name  \
0     spotify:track:7BJz4hamLaRwAcI1KtLHeI            Sin Rumbo   
1     spotify:track:1dm6z1fWB0cErMszU25dy2  Después de la Playa   
2     spotify:track:51FvjPEGKq2zByeeEQ43V9          El Merengue   
3     spotify:track:32b67p4fXwcbGqpjGpUfom    Solo Pienso En Ti   
4     spotify:track:2XmVB1FSy1elvi2O2d8tBj             Merengón   
...                                    ...                  ...   
2009  spotify:track:6LqI79HurtSLxv8ulJohvG                 Papá   
2010  spotify:track:4VgKKuLoLAcgNvJ8d0hKlA      Alzypher Vol. 9   
2011  spotify:track:5UUkHi8YrakZKY9gDbtVNz               Toma 4   
2012  spotify:track:5eO9DgpkoSm2k9eOkI14Xg              Uzumaki   
2013  spotify:track:4NTOD6p2LLDYdeHzaxUYxk              Envidia   

        Artist Name                                      Artist Genres  \
0      Eddy Herrera                              [latin pop, tropical]   
1         Bad Bunny            [reggaeton, trap

In [19]:
import pandas as pd

# Create an empty DataFrame to store the extracted audio features
audio_features_df = pd.DataFrame(columns=["Track URI", "Speechiness", "Tempo", "Valence", "Duration_ms", "Instrumentalness", "Energy", "Danceability"])

# Iterate over the rows of the original DataFrame
for index, row in df.iterrows():
    track_uri = row["Track URI"]
    
    # Retrieve the audio features for the track URI
    audio_features = sp.audio_features(track_uri)[0]
    
    # Extract the desired audio features from the response
    speechiness = audio_features["speechiness"]
    tempo = audio_features["tempo"]
    valence = audio_features["valence"]
    duration_ms = audio_features["duration_ms"]
    instrumentalness = audio_features["instrumentalness"]
    energy = audio_features["energy"]
    danceability = audio_features["danceability"]
    
    # Append the extracted audio features to the new DataFrame
    audio_features_df = audio_features_df.append({
        "Track URI": track_uri,
        "Speechiness": speechiness,
        "Tempo": tempo,
        "Valence": valence,
        "Duration_ms": duration_ms,
        "Instrumentalness": instrumentalness,
        "Energy": energy,
        "Danceability": danceability
    }, ignore_index=True)

# Print the new DataFrame with the extracted audio features
print(audio_features_df)


/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_8544/620438079.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  audio_features_df = audio_features_df.append({
/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_8544/620438079.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  audio_features_df = audio_features_df.append({
/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_8544/620438079.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  audio_features_df = audio_features_df.append({
/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_8544/620438079.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

                                 Track URI  Speechiness    Tempo  Valence  \
0     spotify:track:7BJz4hamLaRwAcI1KtLHeI       0.0590  123.084    0.796   
1     spotify:track:1dm6z1fWB0cErMszU25dy2       0.3070   78.293    0.607   
2     spotify:track:51FvjPEGKq2zByeeEQ43V9       0.0442  124.011    0.698   
3     spotify:track:32b67p4fXwcbGqpjGpUfom       0.1930  117.149    0.871   
4     spotify:track:2XmVB1FSy1elvi2O2d8tBj       0.0460  135.009    0.547   
...                                    ...          ...      ...      ...   
2009  spotify:track:6LqI79HurtSLxv8ulJohvG       0.0883  108.048    0.427   
2010  spotify:track:4VgKKuLoLAcgNvJ8d0hKlA       0.3910   77.326    0.641   
2011  spotify:track:5UUkHi8YrakZKY9gDbtVNz       0.4100   79.975    0.671   
2012  spotify:track:5eO9DgpkoSm2k9eOkI14Xg       0.2000   61.350    0.466   
2013  spotify:track:4NTOD6p2LLDYdeHzaxUYxk       0.2780  149.961    0.298   

     Duration_ms  Instrumentalness  Energy  Danceability  
0         211099

In [22]:
audio_features_df.to_excel('audio_features.xlsx', index=False)


In [18]:
df.to_excel('spotify_song_data.xlsx', index=False)


# Using the Lyrics genius API to collect the song lyrics

In [24]:
pip install lyricsgenius


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 771.9 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [57]:
import time
import lyricsgenius
import requests.exceptions

genius = lyricsgenius.Genius('0Yxq8biFr7xPzVzo2j7d2-F17VArymC-x6gNzP_FXWFXcow0IeT7678rrXoXuhdJ')

In [58]:
lyrics_list = []

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    song_name = row['Track Name']
    artist_name = row['Artist Name']

    retries = 3  # Number of retries
    delay = 2  # Delay between retries in seconds

    for attempt in range(retries):
        try:
            # Search for the song lyrics
            song = genius.search_song(song_name, artist_name)

            # Check if lyrics were found
            if song is not None:
                lyrics = song.lyrics
            else:
                lyrics = "No lyrics available"

            # Append the lyrics to the list
            lyrics_list.append(lyrics)

            break  # Break out of the retry loop if successful

        except requests.exceptions.Timeout:
            # Handle timeout error
            print(f"Timeout occurred for '{song_name}' by '{artist_name}'. Retrying...")
            time.sleep(delay)  # Delay before retrying

        except Exception as e:
            # Handle other exceptions
            print(f"Error occurred for '{song_name}' by '{artist_name}': {str(e)}")
            lyrics_list.append("No lyrics available")
            break  # Break out of the retry loop if error occurs

    else:
        # Retry limit exceeded
        print(f"Retry limit exceeded for '{song_name}' by '{artist_name}'. Skipping...")
        lyrics_list.append("No lyrics available")

# Add the lyrics list as a new column in the DataFrame
df['Lyrics'] = lyrics_list

Searching for "Sin Rumbo" by Eddy Herrera...
Done.
Searching for "Después de la Playa" by Bad Bunny...
Done.
Searching for "El Merengue" by Marshmello...
Done.
Searching for "Solo Pienso En Ti" by Elvis Crespo...
No results found for: 'Solo Pienso En Ti Elvis Crespo'
Searching for "Merengón" by La Cassandra...
No results found for: 'Merengón La Cassandra'
Searching for "Tiburon" by Proyecto Uno...
Done.
Searching for "Mujeriego" by Ryan Castro...
Done.
Searching for "Arranca (feat. Omega)" by Becky G...
Done.
Searching for "La Vaca" by Mala Fe...
Done.
Searching for "El Niágara en Bicicleta" by Juan Luis Guerra 4.40...
No results found for: 'El Niágara en Bicicleta Juan Luis Guerra 4.40'
Searching for "Loma de Cayenas" by Vicente Garcia...
Done.
Searching for "Tu Eres Ajena" by Eddy Herrera...
Done.
Searching for "Kiliki Taka Ti" by Toño Rosario...
No results found for: 'Kiliki Taka Ti Toño Rosario'
Searching for "Tu Sonrisa" by Elvis Crespo...
Done.
Searching for "Niña Bonita - Remast

In [60]:
df.to_excel('spotify_song_data_lyrics.xlsx', index=False)


# Combining the datasets and cleaning song lyrics

In [2]:
import pandas as pd


In [5]:
dataframe1 = pd.read_excel('spotify_song_data_lyrics.xlsx')
dataframe2 = pd.read_excel('audio_features.xlsx')


In [8]:
dataframe1["Speechiness"] = dataframe2['Speechiness']
dataframe1["Tempo"] = dataframe2['Tempo']
dataframe1["Valence"] = dataframe2['Valence']
dataframe1["Duration_ms"] = dataframe2['Duration_ms']
dataframe1["Instrumentalness"] = dataframe2['Instrumentalness']
dataframe1["Energy"] = dataframe2['Energy']
dataframe1["Danceability"] = dataframe2['Danceability']


In [10]:
#changes the display settings so it's possible to see all the lyrics in the columns 
pd.set_option('display.max_colwidth', None)


In [11]:
#this removes every instance of square brackets (inc the brackets themselves) this is a simpler way of deleting words like 'Outro, Intro', 'Choros' etc. And no lyrics are contained within sqaure brackets
dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'\[.*?\]', '')
#removes all instances of '/n' and '/n/n' 
dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'[\n]+', '')
#Manually inserts full stops before every capitalised letter (this poses some limitations e.g., names are capialised but they don't mark the start of a new sentence however this is not necessary for my anaylsis and inserting full stops makes the text easier to read)
dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'(\b\w+)\s*([A-Z])', r'\1. \2')
#This separates Lyrics from the word before 
dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'(\w)([A-Z][a-z]+)', r'\1 \2')
#this removes everything up until and including the word 'Lyrics'
dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'^.*?Lyrics:?\s*', '')
#removes the rows where no lyrics were available 
dataframe1 = dataframe1[dataframe1['Lyrics'] != 'No lyrics available']
#removes 'Embed' at the end of each set of lyrics
dataframe1['Lyrics'] = dataframe1['Lyrics'].str.extract(r'^(.*?)\s?\bEmbed\b', expand=False)


/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_2905/1961338632.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'\[.*?\]', '')
/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_2905/1961338632.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'[\n]+', '')
/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_2905/1961338632.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'(\b\w+)\s*([A-Z])', r'\1. \2')
/var/folders/8x/6vr59jw17ylfldrmvc065_7r0000gn/T/ipykernel_2905/1961338632.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe1['Lyrics'] = dataframe1['Lyrics'].str.replace(r'(\w

In [13]:
dataframe1.to_excel('audio_features_and_clean_lyrics.xlsx', index=False)
